In [2]:
import pandas as pd
import numpy as np

import sklearn
#import ensemble
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error,balanced_accuracy_score,recall_score
import random
import lightgbm as lgb

In [3]:
data = pd.read_parquet("Data/Training/Amex_Campus_Challenge_Train.parquet")

In [4]:
randomlist = random.sample(list(data.customer.unique()),int(0.3*len(data.customer.unique())))

In [5]:
testdata = data[data['customer'].isin(randomlist)]

In [6]:
#select rows whose column value is not in a list
traindata = data[~data['customer'].isin(randomlist)]

In [7]:
datarec = traindata[traindata['ind_recommended']==1].drop(['ind_recommended'], axis=1)
datanotrec = traindata[traindata['ind_recommended']==0].drop(['ind_recommended'], axis=1)

In [8]:
def confusion_matrix_2col(df, col1, col2):
    return df.groupby([col1, col2]).size().unstack(fill_value=0)

confusion_matrix_2col(traindata, 'ind_recommended', 'activation')

activation,0,1
ind_recommended,,
0,7436319,42053
1,1077971,6940


In [9]:
def get_model(df):
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop(['customer','merchant','activation'],axis=1), df['activation'], test_size=0.1, random_state=1)
    rec = lgb.LGBMRegressor()
    rec.fit(X_train, y_train)
    y_pred = rec.predict(X_test)
    # print("Balanced Accuracy :",balanced_accuracy_score(y_test, y_pred)," recall :",recall_score(y_test, y_pred))
    print("mse : ",mean_squared_error(y_test, y_pred))
    return rec

In [10]:
datarec['customer'].groupby(datarec['activation']).count()

activation
0    1077971
1       6940
Name: customer, dtype: int64

In [11]:
rec = get_model(datarec.drop(datarec[datarec['activation'] == 0].sample(n=1000000).index))

mse :  0.0472097513894631


In [12]:
datanotrec['customer'].groupby(datanotrec['activation']).count()

activation
0    7436319
1      42053
Name: customer, dtype: int64

In [13]:
nrec = get_model(datanotrec.drop(datanotrec[datanotrec['activation'] == 0].sample(n=7302748).index))

mse :  0.07983543340266368


In [14]:
a = testdata[['ind_recommended','activation','customer','merchant']]
a.to_csv('d_test.csv', index=False)

In [15]:
try:
    testdata.drop(['rec','nrec','predicted_score'],axis=1,inplace=True)
except:
    pass

In [16]:
rec_list = rec.predict(testdata.drop(['ind_recommended','activation','customer','merchant'],axis=1))
nrec_list = nrec.predict(testdata.drop(['ind_recommended','activation','customer','merchant'],axis=1))

In [17]:
#add column to df 
testdata['rec'] = list(rec_list)
testdata['nrec'] = list(nrec_list)

/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_77226/89867839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdata['rec'] = list(rec_list)
/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_77226/89867839.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdata['nrec'] = list(nrec_list)


In [18]:
# testdata['predicted_score'] = testdata['rec']*(abs(1-testdata['nrec']))
testdata['predicted_score'] = testdata['rec']-testdata['nrec']

/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_77226/1127479097.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdata['predicted_score'] = testdata['rec']-testdata['nrec']


In [19]:
testdata

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant,rec,nrec,predicted_score
11,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.750000,0.000000,2.333333,25.0,4.970615,129516,87423,0.000055,-0.006038,0.006093
20,0,0,1.0,115.591212,33.371818,11.0,367.09,11.0,11.0,11.0,...,62.727500,85.323982,1.500000,378.0,1.589301,34662,63792,0.024084,0.261400,-0.237316
22,1,0,NaN,24.482500,93.407333,7.0,1401.11,15.0,10.0,7.0,...,65.370000,99.873150,2.333333,258.0,18.607740,120246,298378,0.032977,0.100152,-0.067175
25,0,0,NaN,63.175703,91.473421,22.0,3475.99,38.0,24.0,25.0,...,265.500000,100.000000,14.166667,157.0,4.805388,377953,316397,0.002488,0.047574,-0.045085
26,0,0,NaN,24.340000,NaN,NaN,NaN,NaN,NaN,2.0,...,25.780000,20.784777,6.000000,28.0,6.295288,147492,73587,0.025598,0.116526,-0.090928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12229960,0,1,2.0,336.494865,111.918182,34.0,6155.50,55.0,50.0,8.0,...,60.000000,54.607556,0.333333,558.0,2.794109,22528,517279,0.192717,0.504274,-0.311557
12229961,0,1,2.0,62.937706,337.189524,16.0,7080.98,21.0,19.0,20.0,...,61.510000,82.517005,3.000000,353.0,2.350313,22603,189934,0.467700,0.867286,-0.399586
12229973,0,1,10.0,41.706875,13.162500,9.0,368.55,28.0,19.0,5.0,...,17.568750,77.327287,3.166667,510.0,0.109332,23040,564879,1.036137,1.019830,0.016306
12229975,0,1,NaN,61.496000,107.744375,5.0,1723.91,16.0,10.0,13.0,...,85.285000,100.000000,4.666667,612.0,3.300739,23181,259023,0.488474,0.898783,-0.410309


In [20]:
testdata[['customer','predicted_score','merchant']].to_csv('d_pred.csv',index=False)

In [21]:
evaldata = pd.read_parquet('Data/Evaluation/Amex_Campus_Challenge_Eval_Round1.parquet')

In [22]:
evaldata

,distance_05,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,customer_spend_03,...,merchant_profile_02,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,customer,merchant
0,1.621171,NaN,112.334000,80.552500,9.0,966.63,12.0,10.0,4.0,41.0,...,0.437500,26299.0,4777.0,0.800000,33.300,72.268283,7.000000,423.0,467915,599167
1,2.441944,NaN,112.334000,NaN,NaN,NaN,NaN,NaN,4.0,41.0,...,0.397059,7122.0,4803.0,0.800000,793.290,72.268283,7.000000,423.0,467915,686617
2,2.438082,NaN,112.334000,71.192500,3.0,284.77,4.0,4.0,4.0,41.0,...,NaN,7222.0,14860.0,0.800000,100.000,72.268283,7.000000,423.0,467915,829193
3,2.072182,NaN,112.334000,NaN,NaN,NaN,NaN,NaN,4.0,41.0,...,0.142857,11410.0,11968.0,0.800000,252.380,72.268283,7.000000,423.0,467915,1077034
4,2.380853,NaN,302.792500,NaN,NaN,NaN,NaN,NaN,3.0,37.0,...,0.100000,1847.0,5842.0,0.800000,87.500,72.268283,7.000000,423.0,467915,876647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8496461,22.197181,NaN,65.181875,171.463500,8.0,3429.27,20.0,16.0,21.0,94.0,...,NaN,25017.0,29209.0,0.000000,49.500,92.116548,1.666667,270.0,464447,644485
8496462,17.812486,NaN,15.680000,11.095556,5.0,99.86,9.0,8.0,2.0,14.0,...,0.452830,39731.0,3529.0,0.733333,32.440,34.500012,0.166667,24.0,522564,802309
8496463,8.299482,NaN,61.723409,NaN,NaN,NaN,NaN,NaN,11.0,112.0,...,0.270833,2994.0,7534.0,1.000000,198.500,50.484705,0.166667,390.0,465030,741321
8496464,4.665418,NaN,270.908125,31.177692,4.0,405.31,13.0,6.0,10.0,28.0,...,0.250000,423.0,241.0,0.916667,33.000,86.240872,0.333333,310.0,495259,625845


In [23]:
rec_list = rec.predict(evaldata.drop(['customer','merchant'],axis=1))
nrec_list = nrec.predict(evaldata.drop(['customer','merchant'],axis=1))

: 

: 